In [1]:
import codigo
from codigo import utils
from codigo import model_utils

In [2]:
import pandas as pd
import numpy as np
pd.set_option('mode.chained_assignment',None)
import matplotlib.pyplot as plt
import seaborn as sns
import math
from pickle import dump

%matplotlib inline

# Importar Dataset depurado

In [3]:
df_Inicial = pd.read_csv(r'dataset/generado/datasetDepurado.csv')

# Revisamos las columnas que contiene el Dataset

In [4]:
df_Inicial.head()

,ID,Date,Glucose level
0,LIB193263,2020-06-09 19:08:00,99.0
1,LIB193263,2020-06-09 19:23:00,92.0
2,LIB193263,2020-06-09 19:38:00,86.0
3,LIB193263,2020-06-09 19:53:00,85.0
4,LIB193263,2020-06-09 20:08:00,85.0


In [5]:
df_Inicial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2966685 entries, 0 to 2966684
Data columns (total 3 columns):
 #   Column         Dtype  
---  ------         -----  
 0   ID             object 
 1   Date           object 
 2   Glucose level  float64
dtypes: float64(1), object(2)
memory usage: 67.9+ MB


### Cambiamos el formato de la columna Date de object a datetime64

In [24]:
df_Inicial['Date']=pd.to_datetime(df_Inicial['Date'])

In [7]:
df_Inicial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2966685 entries, 0 to 2966684
Data columns (total 3 columns):
 #   Column         Dtype         
---  ------         -----         
 0   ID             object        
 1   Date           datetime64[ns]
 2   Glucose level  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 67.9+ MB


In [8]:
df_Inicial['Date'].min()

Timestamp('2018-01-06 00:31:00')

In [9]:
df_Inicial['Date'].max()

Timestamp('2022-03-20 23:58:00')

# Iniciamos Análisis Exploratorio

## Sobre el conjunto de datos depurado

In [30]:
from pandas_profiling import ProfileReport
profile = ProfileReport(df_Inicial)

In [31]:
profile.to_file("analisisExploratorio/datasetDepurado.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Sobre el conjunto de un paciente

In [10]:
from pickle import load
scalerGlucosa = load(open('dataset/generado/scalerGlucosa.scaler', 'rb'))
scalerHours = load(open('dataset/generado/scalerHours.scaler', 'rb'))
scalerMin = load(open('dataset/generado/scalerMin.scaler', 'rb'))
scalerPodId = load(open('dataset/generado/scalerPodId.scaler', 'rb'))
scalerLevelId = load(open('dataset/generado/scalerLevelId.scaler', 'rb'))

In [11]:
pacienteID="LIB193277"
paciente_data = utils.getDataPatient(df_Inicial,pacienteID,'15min',False)
paciente_data=utils.fillNullData(paciente_data,'interpolate_linear')
paciente_data=utils.generateNewColumns(paciente_data,scalerLevelId,scalerHours,scalerMin,scalerPodId,scalerGlucosa,False)

In [19]:
paciente_data

,Glucose level,level_label,level_id,hour,min,pod_label,pod_id
Date,,,,,,,
2018-06-12 20:15:00,156.0,euglycemia,0,20,15,Eve,4
2018-06-12 20:30:00,163.0,euglycemia,0,20,30,Eve,4
2018-06-12 20:45:00,158.0,euglycemia,0,20,45,Eve,4
2018-06-12 21:00:00,167.0,euglycemia,0,21,0,Night,5
2018-06-12 21:15:00,139.0,euglycemia,0,21,15,Night,5
...,...,...,...,...,...,...,...
2022-03-18 10:45:00,153.0,euglycemia,0,10,45,Morning,2
2022-03-18 11:00:00,144.0,euglycemia,0,11,0,Morning,2
2022-03-18 11:15:00,134.0,euglycemia,0,11,15,Morning,2


In [13]:
from pandas_profiling import ProfileReport
profile = ProfileReport(paciente_data, tsmode=True, sortby="Date")

In [14]:
profile.to_file("analisisExploratorio/paciente_LIB193277.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Muestreo y Selección

## Contamos los registros de cada paciente

In [ ]:
dfPacienteCount= pd.DataFrame(df.ID.value_counts().sort_values(ascending=False))
dfPacienteCount.columns = ['Freq']
total=dfPacienteCount['Freq'].sum()
dfPacienteCount['Porcentaje_Acum']=(dfPacienteCount['Freq'].cumsum()/total)*100
dfPacienteCount

In [ ]:
dfPacienteCount.tail(50)

In [ ]:
dfPacienteCount['Freq'].plot.bar(figsize=(15, 5))

In [ ]:
dfPacienteCount['Porcentaje_Acum'].plot.bar(figsize=(15, 5))

In [ ]:
values_list=['LIB193321','LIB193399','LIB193271']
dfIDPacientesTrain=dfPacienteCount[~dfPacienteCount.index.isin(values_list)]

## Seleccionamos los pacientes que representan el 90% del dataset

In [ ]:
porcentajeTotalDataSet=90

In [ ]:
dfIDPacientesTrain = dfPacienteCount[dfPacienteCount['Porcentaje_Acum'] <=porcentajeTotalDataSet ]
dfIDPacientesTrain.tail()

In [ ]:
dfIDPacientesTrain.info()

In [ ]:
dfIDPacientesTrain.sum()

In [ ]:
dfIDPacientesTest = dfPacienteCount[dfPacienteCount['Porcentaje_Acum'] >porcentajeTotalDataSet ]
dfIDPacientesTest.head()

In [ ]:
dfIDPacientesTest.info()

### Guardamos los IDs de Train y Test

In [ ]:
dfIDPacientesTrain['ID']=dfIDPacientesTrain.index
dfIDPacientesTrain['ID'].to_csv('dataset/generado/IDs_Train.csv', index=False) 
dfIDPacientesTrain['ID'].count()

In [ ]:
dfIDPacientesTest['ID']=dfIDPacientesTest.index
dfIDPacientesTest['ID'].to_csv('dataset/generado/IDs_Test.csv', index=False) 
dfIDPacientesTest['ID'].count()

## Limpieza de Datos

### Analizamos los datos de 1 paciente con las muestras ordenadas cada 15 min

In [ ]:
pacienteID="LIB193277"
paciente = utils.getDataPatient(df,pacienteID,'15min')
paciente

Realizamos un análisis exploratorio

In [ ]:
profile = ProfileReport(paciente, title="Pandas Profiling Report",
    missing_diagrams=None,
    correlations=None, interactions=None,samples=None)
profile.to_file("analisisExploratorio/analisis_paciente_ID_LIB193277.html")

Obtenemos los registros faltantes

In [ ]:
paciente_Null=paciente.isnull()
paciente_Null.sum()

Se observa que no todos los registros fueron tomados cada 15 min, y existen datos faltantes

In [ ]:
paciente_Null.tail()

Pasamos a excel para analizar en detalle esos datos faltantes

In [ ]:
paciente.to_excel("analisisExploratorio/paciente_ID_LIB193277.xlsx")  

In [ ]:
paciente['Glucose level'].plot(figsize=(20, 10))

In [ ]:
import missingno as msno
msno.matrix(paciente)

In [ ]:
paciente_data_15['hour'].min()

#  Evaluación de caracteristicas

In [ ]:
features=['Glucose level','hour','min','pod_id','level_id']

##  Variable Yt

In [ ]:
paciente_data_15['Yt'].plot(figsize=(10, 5))

##  Análisis de correlación

In [ ]:
paciente_df=paciente_data_15[features].copy()
correlated_features = set()
correlation_matrix = paciente_df.corr() 
#obtenemos las caracteristicas que tienen correlacion >0.5 con respecto a la variable objetivo "Glucose level"
correlated_features = correlation_matrix.index[abs(correlation_matrix["Glucose level"])>0.5]
            
correlation_matrix

In [ ]:
arrayLevel=[-1,0,1]
dfLevel= pd.DataFrame(arrayLevel)
dfLevel.columns = ['levelid']
dfLevel

In [ ]:
scalerLevelId = MinMaxScaler()

arrayLevel=scalerLevelId.fit_transform(dfLevel[['levelid']].values)
dump(scalerLevelId, open('dataset/generado/scalerLevelId.scaler', 'wb'))
arrayLevel

In [ ]:
arrayLevelT=scalerLevelId.transform([[-1]])
arrayLevelT[0]

# Estrategia de creación de caracteristicas

In [ ]:
df

In [ ]:
paciente_data_15 = utils.getDataPatient(df,pacienteID,'15min',False)

paciente_data_15['level_label'] = paciente_data_15['Glucose level'].apply(utils.label_LevelBG)
paciente_data_15['level_id'] = paciente_data_15['level_label'].apply(utils.id_LevelBG)
paciente_data_15['level_id'] = scalerLevelId.transform(paciente_data_15[['level_id']].values)
paciente_data_15['Glucose level'] = scalerGlucosa.transform(paciente_data_15[['Glucose level']].values)

paciente_data_15=utils.fillNullData(paciente_data_15,'interpolate_linear')

paciente_data_15=utils.generateNewColumns(paciente_data_15,scalerHours,scalerMin)
paciente_data_15['pod_id'] = scalerPodId.transform(paciente_data_15[['pod_id']].values)

paciente_data_15

In [ ]:
paciente_data_15['hour'].max()

In [ ]:
correlated_features

In [ ]:
plt.figure(figsize=(10, 5))
sns.heatmap(correlation_matrix[features].corr(),  vmax=.9, square=True, annot=True,cmap='coolwarm')
plt.show()

##  Análisis de balanceo de clases

### Clase: Hora del día

In [ ]:
paciente_data_15['hour'].value_counts().plot.bar( rot=0,figsize=(10, 3))

### Clase: Parte del día

In [ ]:
paciente_data_15['pod_label'].value_counts().plot.bar( rot=0,figsize=(10, 3))

### Clase: Niveles de Glucosa

In [ ]:
paciente_data_15['level_label'].value_counts().plot.bar( rot=0,figsize=(10, 4))

In [ ]:
paciente_data_15['level_label'].value_counts().plot.pie(y='level_label',legend=False,autopct='%1.1f%%',figsize=(10, 4))

In [ ]:
paciente_data_15['level_label'].value_counts()

# Generación de Dataset General de entrenamiento, completo y limpio

In [ ]:
dfIDPacientesTrain = pd.read_csv(r'dataset/generado/IDs_Train.csv')
dfIDPacientesTrain

In [ ]:
min=15
print(str(min)+'min')

In [ ]:
dfIDPacientesTrain

In [ ]:
df

In [ ]:
# Generamos train validation and test 
# ratio of training data set (85%, 10% for validation and 5% for testing)
train_share = 0.85
#ration of validation data set (10%)
val_share = 0.95 
lag=30
n_ahead=1


dfGen,array_Xtrain,array_Ytrain,array_Xval,array_Yval,array_Xtest,array_Ytest=model_utils.GenDataSet(df,features,dfIDPacientesTrain['ID'],15,train_share,val_share,lag,n_ahead,scalerHours,scalerMin,scalerGlucosa,scalerPodId,scalerLevelId)



In [ ]:
len(array_Xtrain)

In [ ]:
len(array_Ytrain)

In [ ]:
len(array_Xval)

In [ ]:
len(array_Xtest)

In [ ]:
array_Ytest

In [ ]:
dfGen

In [ ]:
dump(dfGen, open('dataset/generado/dfGen.dataset', 'wb'))

In [ ]:
Yscaled=scalerGlucosa.inverse_transform([dfGen['Glucose level'].values])

In [ ]:
df_Y_scaled = pd.DataFrame(Yscaled[0], columns=['Glucose level']) 
df_Y_scaled['Nivel'] = df_Y_scaled['Glucose level'].apply(utils.label_LevelBG) 
df_Y_scaled['Nivel'].value_counts().plot.pie(title='Distribución de Niveles de Glucosa',y='Nivel',legend=False,autopct='%1.1f%%',figsize=(10, 4))
plt.show()

In [ ]:
df_Y_scaled['Nivel'].value_counts()

In [ ]:
dfGen['level_label']= dfGen['level_id'].apply(utils.id_LevelBG_inverse_transform,scaler=scalerLevelId)
dfGen['level_label'].value_counts().plot.bar( rot=0,figsize=(10, 4))

In [ ]:
dfGen['level_label'].value_counts()

In [ ]:
dfGen['level_label'].value_counts().plot.pie(y='level_label',legend=False,autopct='%1.1f%%',figsize=(10, 4))

In [ ]:
dfGen['pod_label']= dfGen['pod_id'].apply(utils.id_partOfDay_inverse_transform,scaler=scalerPodId)
dfGen['pod_label'].value_counts().plot.bar( rot=0,figsize=(10, 5))

In [ ]:
dfGen['hour'].value_counts().plot.bar( rot=0,figsize=(10, 3))